In [1]:
# Copyright 2015 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Trains and Evaluates the MNIST network using a feed dictionary."""
# pylint: disable=missing-docstring
import sys
sys.path.append('../../')
import os
import time
import numpy
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import input_test
import math
import numpy as np
from i3d import InceptionI3d
from utils import *

# Basic model parameters as external flags.
flags = tf.app.flags
gpu_num = 1
flags.DEFINE_integer('batch_size', 1, 'Batch size.')
flags.DEFINE_integer('num_frame_per_clib', 64, 'Nummber of frames per clib')
flags.DEFINE_integer('crop_size', 224, 'Crop_size')
flags.DEFINE_integer('rgb_channels', 3, 'Channels for input')
flags.DEFINE_integer('classics', 8, 'The num of class')
FLAGS = flags.FLAGS

os.environ["CUDA_VISIBLE_DEVICES"]="2"
outfile = '/media/ee401_2/SPML/HMDB_rgb_feat_hr/test/'
if not os.path.exists(outfile):
    os.makedirs(outfile)


def run_training():
    # Get the sets of images and labels for training, validation, and
    # Tell TensorFlow that the model will be built into the default Graph.
    pre_model_save_dir = "/home/ee401_2/Documents/I3D-Tensorflow-master/experiments/hmdb-51/model_HR/HR_rgb_imagenet_10000_6_64_0.0001_decay/"
    if not os.path.exists(pre_model_save_dir):
        print('WARNING !!!!! check the path model')
        exit()
    else:
        test_list_file = '../../list/hmdb_list/testlist1_hr.list'
        file = list(open(test_list_file, 'r'))
        num_test_videos = len(file)
        print("Number of test videos={}".format(num_test_videos))
        with tf.Graph().as_default():
            rgb_images_placeholder, _, labels_placeholder, is_training = placeholder_inputs(
                            FLAGS.batch_size * gpu_num,
                            FLAGS.num_frame_per_clib,
                            FLAGS.crop_size,
                            FLAGS.rgb_channels
                            )

            with tf.variable_scope('RGB'):
                logit, _ = InceptionI3d(
                                    num_classes=FLAGS.classics,
                                    spatial_squeeze=True,
                                    final_endpoint='Mixed_5c',
                                    name='inception_i3d'
                                    )(rgb_images_placeholder, is_training)
            norm_score = tf.nn.softmax(logit)

            # Create a saver for writing training checkpoints.
            saver = tf.train.Saver()
            init = tf.global_variables_initializer()

            # Create a session for running Ops on the Graph.
            sess = tf.Session(
                            config=tf.ConfigProto(allow_soft_placement=True)
                            )
            sess.run(init)

        ckpt = tf.train.get_checkpoint_state(pre_model_save_dir)
        if ckpt and ckpt.model_checkpoint_path:
            print ("loading checkpoint %s,waiting......" % ckpt.model_checkpoint_path)
            saver.restore(sess, ckpt.model_checkpoint_path)
            print ("load complete!")

        all_steps = num_test_videos
        top1_list = []
        for step in xrange(all_steps):
            start_time = time.time()
            s_index = 0
            predicts = []
            data = {}
            top1 = False
            data_new = np.array([])
            while True:
                val_images, _, val_labels, s_index, is_end = input_test.read_clip_and_label(
                                filename=file[step],
                                batch_size=FLAGS.batch_size * gpu_num,
                                s_index=s_index,
                                num_frames_per_clip=FLAGS.num_frame_per_clib,
                                crop_size=FLAGS.crop_size,
                                )

                data_features = sess.run(logit,
                                   feed_dict={
                                                rgb_images_placeholder: val_images,
                                                labels_placeholder: val_labels,
                                                is_training: False
                                                })

                #print("feat ================")
                #print(data_features.shape)
                #print("feat ================")

                temp = np.mean(data_features, axis=2)
                tmp = np.mean(temp, axis=2)
                tmp = np.squeeze(tmp)

                #print("feat ================")
                #tmp = data_features
                #print(tmp.shape)
                #print("feat ================")


                if data_new.shape[0] == 0:
                    data_new = tmp
                else:
                    data_new = np.concatenate([data_new,tmp])

                labell = np.zeros((1,51))
                labell[0,int(val_labels)] = 1

                #print(labell)

                if is_end:
                    ndata = int(data_new.shape[0])
                    nn = 32
                    if ndata <=nn:
                        data_new = np.concatenate([data_new,data_new],  axis=0)
                        data_new = np.concatenate([data_new,data_new],  axis=0)
                        ndata = int(data_new.shape[0])

                    sampling = ndata/nn
                    sampling = int(sampling)
                    #print(sampling)
                    #print(data_new.shape)
                    ndata2 = data_new[0:nn*sampling,:]
                    ndata3 = ndata2[0::sampling,:]

                    data['feat'] = ndata3
                    data['label'] = labell

                    temp = file[step].split("/")
                    namefile = temp[6].split(" ")
                    print(namefile[0])

                    datasave = outfile+namefile[0]
                    print(datasave)
                    np.save(datasave, data)
                    break


def main(_):
    run_training()


if __name__ == '__main__':
    tf.app.run()


Number of test videos=1530
loading checkpoint /home/ee401_2/Documents/MMSP2019/I3D-Tensorflow/experiments/hmdb-51/model_HR/HR_rgb_imagenet_10000_6_64_0.0001_decay/i3d_hmdb_model-9999,waiting......
INFO:tensorflow:Restoring parameters from /home/ee401_2/Documents/MMSP2019/I3D-Tensorflow/experiments/hmdb-51/model_HR/HR_rgb_imagenet_10000_6_64_0.0001_decay/i3d_hmdb_model-9999
load complete!
Loading a video clip from /home/ee401_2/Documents/HMDB_new/kiss/TVs_Best_Kisses_Top_50_52_to_41_kiss_h_nm_np2_le_goo_1...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/kiss/TVs_Best_Kisses_Top_50_52_to_41_kiss_h_nm_np2_le_goo_1...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/kiss/TVs_Best_Kisses_Top_50_52_to_41_kiss_h_nm_np2_le_goo_1...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/kiss/TVs_Best_Kisses_Top_50_52_to_41_kiss_h_nm_np2_le_goo_1...strat_frame = 192
Loading a video clip from /home/ee401_2/Documents/HMDB_n

Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_1...strat_frame = 384
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_1...strat_frame = 448
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_1...strat_frame = 512
Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_1
/media/ee401_2/SPML/HMDB_rgb_feat_hr/test/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_1
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_2...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_2...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brus

Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Brushing_my_long_hair_brush_hair_u_nm_np1_fr_goo_1...strat_frame = 192
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Brushing_my_long_hair_brush_hair_u_nm_np1_fr_goo_1...strat_frame = 256
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Brushing_my_long_hair_brush_hair_u_nm_np1_fr_goo_1...strat_frame = 320
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Brushing_my_long_hair_brush_hair_u_nm_np1_fr_goo_1...strat_frame = 384
Brushing_my_long_hair_brush_hair_u_nm_np1_fr_goo_1
/media/ee401_2/SPML/HMDB_rgb_feat_hr/test/Brushing_my_long_hair_brush_hair_u_nm_np1_fr_goo_1
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Chanta____THAT_IS_long_blonde_hair_brush_hair_u_cm_np1_fr_goo_2...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Chanta____THAT_IS_long_blonde_hair_brush_hair_u_cm_np1_fr_goo_2...stra

Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Prelinger_HabitPat1954_brush_hair_h_nm_np1_fr_med_26...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Prelinger_HabitPat1954_brush_hair_h_nm_np1_fr_med_26...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Prelinger_HabitPat1954_brush_hair_h_nm_np1_fr_med_26...strat_frame = 192
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Prelinger_HabitPat1954_brush_hair_h_nm_np1_fr_med_26...strat_frame = 256
Prelinger_HabitPat1954_brush_hair_h_nm_np1_fr_med_26
/media/ee401_2/SPML/HMDB_rgb_feat_hr/test/Prelinger_HabitPat1954_brush_hair_h_nm_np1_fr_med_26
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Prelinger_HabitPat1954_brush_hair_u_cm_np1_le_med_10...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Prelinger_HabitPat1954_brush_hair_u_cm_np1_le_med_10...strat_frame = 6

Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/me_brushing_my_hair_lol_i_was_being_silly_brush_hair_u_nm_np1_le_goo_1...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/me_brushing_my_hair_lol_i_was_being_silly_brush_hair_u_nm_np1_le_goo_1...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/me_brushing_my_hair_lol_i_was_being_silly_brush_hair_u_nm_np1_le_goo_1...strat_frame = 192
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/me_brushing_my_hair_lol_i_was_being_silly_brush_hair_u_nm_np1_le_goo_1...strat_frame = 256
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/me_brushing_my_hair_lol_i_was_being_silly_brush_hair_u_nm_np1_le_goo_1...strat_frame = 320
me_brushing_my_hair_lol_i_was_being_silly_brush_hair_u_nm_np1_le_goo_1
/media/ee401_2/SPML/HMDB_rgb_feat_hr/test/me_brushing_my_hair_lol_i_was_being_silly_brush_hair_u_nm_np1_le_goo_1
Loading a vi

...

Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/Bush_Wave_vs__Obama_Wave_and_Which_Commander-in-chief_Salutes_the_Best__You_Decide_wave_u_cm_np1_fr_med_1...strat_frame = 192
Bush_Wave_vs__Obama_Wave_and_Which_Commander-in-chief_Salutes_the_Best__You_Decide_wave_u_cm_np1_fr_med_1
/media/ee401_2/SPML/HMDB_rgb_feat_hr/test/Bush_Wave_vs__Obama_Wave_and_Which_Commander-in-chief_Salutes_the_Best__You_Decide_wave_u_cm_np1_fr_med_1
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/Bush_Wave_vs__Obama_Wave_and_Which_Commander-in-chief_Salutes_the_Best__You_Decide_wave_u_cm_np1_fr_med_10...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/Bush_Wave_vs__Obama_Wave_and_Which_Commander-in-chief_Salutes_the_Best__You_Decide_wave_u_cm_np1_fr_med_10...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/Bush_Wave_vs__Obama_Wave_and_Which_Commander-in-chief_Salutes_the_Best__You_Decide_wave_u_cm_np1_fr_med_10...stra

Bush_Wave_vs__Obama_Wave_and_Which_Commander-in-chief_Salutes_the_Best__You_Decide_wave_u_cm_np1_le_med_6
/media/ee401_2/SPML/HMDB_rgb_feat_hr/test/Bush_Wave_vs__Obama_Wave_and_Which_Commander-in-chief_Salutes_the_Best__You_Decide_wave_u_cm_np1_le_med_6
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/DONNIE_DARKO_wave_u_nm_np1_fr_goo_9...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/DONNIE_DARKO_wave_u_nm_np1_fr_goo_9...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/DONNIE_DARKO_wave_u_nm_np1_fr_goo_9...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/DONNIE_DARKO_wave_u_nm_np1_fr_goo_9...strat_frame = 192
DONNIE_DARKO_wave_u_nm_np1_fr_goo_9
/media/ee401_2/SPML/HMDB_rgb_feat_hr/test/DONNIE_DARKO_wave_u_nm_np1_fr_goo_9
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/DONNIE_DARKO_wave_u_nm_np1_le_goo_8...strat_frame = 0
Loading a video clip from /home/ee401

Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/TrumanShow_wave_f_nm_np1_fr_med_26...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/TrumanShow_wave_f_nm_np1_fr_med_26...strat_frame = 128
TrumanShow_wave_f_nm_np1_fr_med_26
/media/ee401_2/SPML/HMDB_rgb_feat_hr/test/TrumanShow_wave_f_nm_np1_fr_med_26
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/TrumanShow_wave_u_nm_np1_fr_med_0...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/TrumanShow_wave_u_nm_np1_fr_med_0...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/TrumanShow_wave_u_nm_np1_fr_med_0...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/TrumanShow_wave_u_nm_np1_fr_med_0...strat_frame = 192
TrumanShow_wave_u_nm_np1_fr_med_0
/media/ee401_2/SPML/HMDB_rgb_feat_hr/test/TrumanShow_wave_u_nm_np1_fr_med_0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/TrumanSh

SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
import numpy as np
a = np.ones((2,3,4,5))

In [7]:
a[2:]

array([], shape=(0, 3, 4, 5), dtype=float64)

SyntaxError: invalid syntax (<ipython-input-5-0e439c08fc3a>, line 1)